In [1]:
import pandas as pd

bens_cand = pd.read_csv('bem_candidato_2018_BRASIL.csv')
consulta_cand = pd.read_csv('consulta_cand_2018_BRASIL.csv')

In [2]:
bens_cand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93245 entries, 0 to 93244
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   DT_GERACAO             93245 non-null  object
 1   HH_GERACAO             93245 non-null  object
 2   ANO_ELEICAO            93245 non-null  int64 
 3   CD_TIPO_ELEICAO        93245 non-null  int64 
 4   NM_TIPO_ELEICAO        93245 non-null  object
 5   CD_ELEICAO             93245 non-null  int64 
 6   DS_ELEICAO             93245 non-null  object
 7   DT_ELEICAO             93245 non-null  object
 8   SG_UF                  93245 non-null  object
 9   SG_UE                  93245 non-null  object
 10  NM_UE                  93245 non-null  object
 11  SQ_CANDIDATO           93245 non-null  int64 
 12  NR_ORDEM_CANDIDATO     93245 non-null  int64 
 13  CD_TIPO_BEM_CANDIDATO  93245 non-null  int64 
 14  DS_TIPO_BEM_CANDIDATO  93245 non-null  object
 15  DS_BEM_CANDIDATO   

In [3]:
bens_cand.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,SG_UF,SG_UE,NM_UE,SQ_CANDIDATO,NR_ORDEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULTIMA_ATUALIZACAO,HH_ULTIMA_ATUALIZACAO
0,15/11/2018,20:03:31,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,MG,MG,MINAS GERAIS,130000600698,9,79,Outros fundos,Itaú Personnalite DI Evolução,"25538,81",12/11/2018,16:49:35
1,15/11/2018,20:03:31,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,MG,MG,MINAS GERAIS,130000600700,14,21,"Veículo automotor terrestre: caminhão, automóv...","Veiculo Renavam 00324329105, valor historico d...","32500,00",12/11/2018,16:49:09
2,15/11/2018,20:03:31,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,MA,MA,MARANHÃO,100000603924,2,12,Casa,#NULO#,"250000,00",23/10/2018,17:29:33
3,15/11/2018,20:03:31,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,PI,PI,PIAUÍ,180000604104,5,21,"Veículo automotor terrestre: caminhão, automóv...","Kombi, ano 1996, placa LVI 6733","14000,00",15/10/2018,08:22:30
4,15/11/2018,20:03:31,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,PI,PI,PIAUÍ,180000604104,7,21,"Veículo automotor terrestre: caminhão, automóv...","Kombi, ano 1996, placa LVH 8318","7000,00",15/10/2018,08:22:30


In [4]:
consulta_cand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29145 entries, 0 to 29144
Data columns (total 58 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   DT_GERACAO                     29145 non-null  object
 1   HH_GERACAO                     29145 non-null  object
 2   ANO_ELEICAO                    29145 non-null  int64 
 3   CD_TIPO_ELEICAO                29145 non-null  int64 
 4   NM_TIPO_ELEICAO                29145 non-null  object
 5   NR_TURNO                       29145 non-null  int64 
 6   CD_ELEICAO                     29145 non-null  int64 
 7   DS_ELEICAO                     29145 non-null  object
 8   DT_ELEICAO                     29145 non-null  object
 9   TP_ABRANGENCIA                 29145 non-null  object
 10  SG_UF                          29145 non-null  object
 11  SG_UE                          29145 non-null  object
 12  NM_UE                          29145 non-null  object
 13  C

In [5]:
consulta_cand.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,NR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO
0,15/11/2018,20:03:47,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,702,OPERADOR DE APARELHOS DE PRODUÇÃO INDUSTRIAL,0,4,NÃO ELEITO,N,S,-1,6007157820186100000
1,15/11/2018,20:03:47,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,BRANCA,115,ODONTÓLOGO,0,2,ELEITO POR QP,N,S,-1,6007369120186230000
2,15/11/2018,20:03:47,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,BRANCA,999,OUTROS,0,5,SUPLENTE,N,N,-1,6009392620186240000
3,15/11/2018,20:03:47,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PRETA,999,OUTROS,-1,-1,#NULO#,N,S,-1,6010367020186080000
4,15/11/2018,20:03:47,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,277,DEPUTADO,0,2,ELEITO POR QP,S,S,-1,6016877520186090000


In [89]:
# Fundindo os dataframes
bens_consulta = pd.merge(consulta_cand, bens_cand, "inner", "SQ_CANDIDATO")

In [7]:
bens_consulta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94134 entries, 0 to 94133
Data columns (total 76 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   DT_GERACAO_x                   94134 non-null  object
 1   HH_GERACAO_x                   94134 non-null  object
 2   ANO_ELEICAO_x                  94134 non-null  int64 
 3   CD_TIPO_ELEICAO_x              94134 non-null  int64 
 4   NM_TIPO_ELEICAO_x              94134 non-null  object
 5   NR_TURNO                       94134 non-null  int64 
 6   CD_ELEICAO_x                   94134 non-null  int64 
 7   DS_ELEICAO_x                   94134 non-null  object
 8   DT_ELEICAO_x                   94134 non-null  object
 9   TP_ABRANGENCIA                 94134 non-null  object
 10  SG_UF_x                        94134 non-null  object
 11  SG_UE_x                        94134 non-null  object
 12  NM_UE_x                        94134 non-null  object
 13  C

In [90]:
# Limitando escopo para primeiro turno, para candidaturas deferidas, para eleições estaduais no Estado
# de São Paulo e para o cargo de deputado estadual
bens_consulta = bens_consulta[
    (bens_consulta['DS_DETALHE_SITUACAO_CAND'].isin(['DEFERIDO', 'DEFERIDO COM RECURSO'])) & 
    (bens_consulta['NR_TURNO'] == 1) &
    (bens_consulta['SG_UF_x'] == 'SP') &
    (bens_consulta['DS_CARGO'] == 'DEPUTADO ESTADUAL')
]

# Dropando colunas desnecessárias
bens_consulta = bens_consulta.drop(columns=[
    "CD_ELEICAO_x",
    "CD_ELEICAO_y",
    "TP_ABRANGENCIA",
    "CD_TIPO_ELEICAO_x",
    "CD_TIPO_ELEICAO_y",
    "NM_TIPO_ELEICAO_x",
    "NM_TIPO_ELEICAO_y",
    "DT_ELEICAO_x",
    "DT_ELEICAO_y",
    "DS_ELEICAO_x",
    "DS_ELEICAO_y",
    "SG_UF_x",
    "SG_UF_y",
    "SG_UE_x",
    "SG_UE_y",
    "NM_UE_x",
    "NM_UE_y",
    "CD_CARGO",
    "DS_CARGO",
    "NR_PARTIDO",
    "SG_PARTIDO",
    "SQ_COLIGACAO",
    "DS_COMPOSICAO_COLIGACAO",
    "TP_AGREMIACAO",
    "SQ_CANDIDATO",
    "NR_CANDIDATO",
    "NM_CANDIDATO",
    "NM_URNA_CANDIDATO",
    "NM_SOCIAL_CANDIDATO",
    "NR_CPF_CANDIDATO",
    "NM_EMAIL",
    "NR_TITULO_ELEITORAL_CANDIDATO",
    "CD_NACIONALIDADE",
    "CD_MUNICIPIO_NASCIMENTO",
    "NM_MUNICIPIO_NASCIMENTO",
    "DT_NASCIMENTO",
    "CD_GENERO",
    "CD_GRAU_INSTRUCAO",
    "CD_ESTADO_CIVIL",
    "CD_COR_RACA",
    "CD_OCUPACAO",
    "CD_TIPO_BEM_CANDIDATO",
    "DS_TIPO_BEM_CANDIDATO",
    "DS_BEM_CANDIDATO",
    "CD_SITUACAO_CANDIDATURA",
    "DS_SITUACAO_CANDIDATURA",
    "CD_DETALHE_SITUACAO_CAND",
    "DS_DETALHE_SITUACAO_CAND",
    "NR_DESPESA_MAX_CAMPANHA",
    "DT_GERACAO_x",
    "DT_GERACAO_y",
    "HH_GERACAO_x",
    "HH_GERACAO_y",
    "ANO_ELEICAO_x",
    "ANO_ELEICAO_y",
    "NR_TURNO",
    "CD_SIT_TOT_TURNO",
    "ST_DECLARAR_BENS",
    "NR_PROTOCOLO_CANDIDATURA",
    "NR_ORDEM_CANDIDATO",
    "NR_PROCESSO",
    "DT_ULTIMA_ATUALIZACAO",
    "HH_ULTIMA_ATUALIZACAO"
])

In [91]:
bens_consulta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6600 entries, 18 to 94060
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   NM_PARTIDO           6600 non-null   object
 1   NM_COLIGACAO         6600 non-null   object
 2   DS_NACIONALIDADE     6600 non-null   object
 3   SG_UF_NASCIMENTO     6600 non-null   object
 4   NR_IDADE_DATA_POSSE  6600 non-null   int64 
 5   DS_GENERO            6600 non-null   object
 6   DS_GRAU_INSTRUCAO    6600 non-null   object
 7   DS_ESTADO_CIVIL      6600 non-null   object
 8   DS_COR_RACA          6600 non-null   object
 9   DS_OCUPACAO          6600 non-null   object
 10  DS_SIT_TOT_TURNO     6600 non-null   object
 11  ST_REELEICAO         6600 non-null   object
 12  VR_BEM_CANDIDATO     6600 non-null   object
dtypes: int64(1), object(12)
memory usage: 721.9+ KB


In [92]:
# Convertendo valor de declaração para numérico
bens_consulta['VR_BEM_CANDIDATO'] = bens_consulta['VR_BEM_CANDIDATO'].str.replace(',', '.')
bens_consulta['VR_BEM_CANDIDATO'] = bens_consulta['VR_BEM_CANDIDATO'].astype(float)

In [93]:
# Transformando coluna idade em categórica
idade = pd.cut(
    bens_consulta['NR_IDADE_DATA_POSSE'],
    bins=[0, 29, 59, 99],
    labels=['JOVEM', 'ADULTO', 'IDOSO']
)
bens_consulta.insert(4, 'CLASSIFICACAO_ETARIA', idade)
bens_consulta = bens_consulta.drop(columns=['NR_IDADE_DATA_POSSE'])

# Convertendo coluna de categoria para string
bens_consulta['CLASSIFICACAO_ETARIA'] = bens_consulta['CLASSIFICACAO_ETARIA'].astype('object')

In [83]:
bens_consulta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6600 entries, 18 to 94060
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   NM_PARTIDO            6600 non-null   object 
 1   NM_COLIGACAO          6600 non-null   object 
 2   DS_NACIONALIDADE      6600 non-null   object 
 3   SG_UF_NASCIMENTO      6600 non-null   object 
 4   CLASSIFICACAO_ETARIA  6600 non-null   object 
 5   DS_GENERO             6600 non-null   object 
 6   DS_GRAU_INSTRUCAO     6600 non-null   object 
 7   DS_ESTADO_CIVIL       6600 non-null   object 
 8   DS_COR_RACA           6600 non-null   object 
 9   DS_OCUPACAO           6600 non-null   object 
 10  DS_SIT_TOT_TURNO      6600 non-null   object 
 11  ST_REELEICAO          6600 non-null   object 
 12  VR_BEM_CANDIDATO      6600 non-null   float64
dtypes: float64(1), object(12)
memory usage: 721.9+ KB


In [94]:
# Diminuindo o número de profissões
import numpy as np

ocupacao = bens_consulta['DS_OCUPACAO'].value_counts().rename_axis('DS_OCUPACAO').reset_index(name='counts')
profissoes_com_menos_de_10_ocorrencias = ocupacao[ocupacao['counts'] < 10]
profissoes_com_menos_de_10_ocorrencias = profissoes_com_menos_de_10_ocorrencias.iloc[:, 0]

bens_consulta['DS_OCUPACAO'] = np.where(
    bens_consulta['DS_OCUPACAO'].isin(profissoes_com_menos_de_10_ocorrencias),
    'OUTROS',
    bens_consulta['DS_OCUPACAO']
)

In [95]:
# Diminuindo o número de Estados
regioes = {
    'SUL': ['SC', 'PR', 'RS'],
    'NORTE': ['AM', 'AP', 'AC', 'PA', 'RO', 'RR', 'TO'],
    'NORDESTE': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    'CENTRO_OESTE': ['GO', 'MT', 'MS'],
    'SUDESTE': ['ES', 'MG', 'RJ', 'SP']
}

for regiao in regioes.keys():
    bens_consulta['SG_UF_NASCIMENTO'] = np.where(
        bens_consulta['SG_UF_NASCIMENTO'].isin(regioes[regiao]),
        regiao,
        bens_consulta['SG_UF_NASCIMENTO']
    )

In [96]:
# Verificando valores nas colunas
for column in bens_consulta.columns:
    if bens_consulta[column].dtype == object:
        print('\n')
        print(column)
        print('\n')
        print(bens_consulta[column].value_counts())
        print('\n')
        print('------------------------------------------------')
        print('\n')



NM_PARTIDO


PODEMOS                                           667
PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA           544
PARTIDO DA REPÚBLICA                              517
PARTIDO HUMANISTA DA SOLIDARIEDADE                353
PARTIDO SOCIALISTA BRASILEIRO                     352
PARTIDO SOCIAL LIBERAL                            328
PARTIDO DOS TRABALHADORES                         325
PARTIDO NOVO                                      298
MOVIMENTO DEMOCRÁTICO BRASILEIRO                  296
PATRIOTA                                          294
PROGRESSISTAS                                     234
PARTIDO VERDE                                     220
PARTIDO REPUBLICANO DA ORDEM SOCIAL               187
DEMOCRATAS                                        180
AVANTE                                            170
PARTIDO COMUNISTA DO BRASIL                       170
PARTIDO REPUBLICANO BRASILEIRO                    166
PARTIDO SOCIAL DEMOCRÁTICO                        163
REDE SUSTENTA

In [97]:
# Removendo valor estranho de Estado de nascimento
bens_consulta = bens_consulta[bens_consulta['SG_UF_NASCIMENTO'] != 'ZZ']

In [98]:
# Transformando target
import numpy as np

bens_consulta['DS_SIT_TOT_TURNO'] = np.where(
    bens_consulta['DS_SIT_TOT_TURNO'] == 'SUPLENTE',
    'NÃO ELEITO',
    bens_consulta['DS_SIT_TOT_TURNO']
)
bens_consulta['DS_SIT_TOT_TURNO'] = np.where(
    bens_consulta['DS_SIT_TOT_TURNO'].isin(['ELEITO POR QP','ELEITO POR MÉDIA']),
    'ELEITO',
    bens_consulta['DS_SIT_TOT_TURNO']
)
bens_consulta['DS_SIT_TOT_TURNO'].value_counts()

NÃO ELEITO    5681
ELEITO         891
Name: DS_SIT_TOT_TURNO, dtype: int64

In [99]:
from sklearn.preprocessing import OneHotEncoder

# Definindo função para transformar colunas categóricas
def one_hot_encode_feature(data,feature):

    enc = OneHotEncoder()
    
    enc_df = pd.DataFrame(enc.fit_transform(data[[feature]]).toarray())
    enc_df.columns = enc.get_feature_names_out()
    
    new_data = pd.concat([data,enc_df],axis=1)
    new_data.drop(feature,axis=1,inplace=True)
    
    return new_data

In [127]:
# Transformando colunas categóricas
bens_consulta_enc = bens_consulta

for column in bens_consulta_enc.columns:
    if bens_consulta_enc[column].dtype == object:
        bens_consulta_enc = one_hot_encode_feature(bens_consulta_enc, column)

In [128]:
# Removendo valores nulos
bens_consulta_enc = bens_consulta_enc.dropna()

for column in bens_consulta_enc.columns:
    if 'nan' in column:
        bens_consulta_enc = bens_consulta_enc.drop(columns=column)

In [131]:
bens_consulta_enc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 563 entries, 18 to 6563
Columns: 197 entries, NR_IDADE_DATA_POSSE to ST_REELEICAO_S
dtypes: float64(197)
memory usage: 870.9 KB
